In [10]:
import pandas as pd
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [11]:
nltk.download('stopwords')
nltk.download('punkt')
df = pd.read_csv('Train.csv')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
df = df[['text']]
df

,text
0,I grew up (b. 1965) watching and loving the Th...
1,"When I put this movie in my DVD player, and sa..."
2,Why do people who do not know what a particula...
3,Even though I have great interest in Biblical ...
4,Im a die hard Dads Army fan and nothing will e...
...,...
39995,"""Western Union"" is something of a forgotten cl..."
39996,This movie is an incredible piece of work. It ...
39997,My wife and I watched this movie because we pl...
39998,"When I first watched Flatliners, I was amazed...."


In [13]:
def preprocess_text(row):
# Convert to lowercase
    text = row.lower()
    # Remove numbers
    text = ''.join([i for i in text if not i.isdigit()])
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return tokens


In [14]:
df.head()
df['tokenized_sents'] = df['text'].apply(preprocess_text)
df.head()

,text,tokenized_sents
0,I grew up (b. 1965) watching and loving the Th...,"[grew, b, watching, loving, thunderbirds, mate..."
1,"When I put this movie in my DVD player, and sa...","[put, movie, dvd, player, sat, coke, chips, ex..."
2,Why do people who do not know what a particula...,"[people, know, particular, time, past, like, f..."
3,Even though I have great interest in Biblical ...,"[even, though, great, interest, biblical, movi..."
4,Im a die hard Dads Army fan and nothing will e...,"[im, die, hard, dads, army, fan, nothing, ever..."


In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# Sample input text
input_text = input("Enter your query: ")
# Tokenized sentences from your dataset
tokenized_sentences = df['tokenized_sents'].tolist()
# Convert tokenized sentences into strings
tokenized_sentences = [" ".join(tokens) for tokens in tokenized_sentences]
# Convert input text into a string if it's a list
if isinstance(input_text, list):
    input_text = " ".join(input_text)
# Convert tokenized sentences and input text into numerical vectors using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(tokenized_sentences + [input_text])
# Compute cosine similarity between input text vector and each vector in the dataset
cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])


In [20]:
# Get indices of most similar items
most_similar_indices = cosine_similarities.argsort()[0][::-1][:5] # Get top 5
#indices
# Create a DataFrame to store top 5 similar products and their cosine
#similarities
top_similar_df = pd.DataFrame(columns=['Title', 'Cosine Similarity'])
# Fill the DataFrame with data
for idx in most_similar_indices:
    text = df.loc[idx, 'text']
    similarity = cosine_similarities[0][idx]
    top_similar_df.loc[len(top_similar_df)] = [text, similarity]
# Display the DataFrame
print(f"\nTop 5 similar products for {input_text}:")
print(top_similar_df)




Top 5 similar products for Bags:
                                               Title  Cosine Similarity
0  I rented this because I'm a bit weary of '80s ...           0.187295
1  I went in not knowing anything about this movi...           0.181219
2  I was fascinated as to how truly bad this movi...           0.173942
3  OK, I am a sucker. I loved it. I had no expect...           0.156673
4  If you're a layman interested in quantum theor...           0.155651


In [26]:
# %%
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import datetime

In [27]:
# %%
model = Word2Vec(sentences=df['tokenized_sents'], vector_size=100, window=5,
min_count=1, sg=0)


In [23]:
vector = model.wv['bag']
print(vector)




[-0.12780282  0.9071737  -0.2354934  -0.5303825   1.1827406   0.5659059
  0.2964308   0.917051   -0.15370122  0.7271319  -0.9855236  -0.26626933
 -0.00430388  0.08617382  0.88665324 -0.79872483  0.08847836  0.29003546
 -0.44187838 -0.8943349   0.34208918  0.8010406   0.44939512 -0.09659745
  0.04627722  0.0173268  -0.73725015 -0.52461576 -0.14174932 -0.7197496
  0.19815166  0.08127058 -0.57616323  0.38616112  0.19289108  0.56673926
 -0.27318165 -0.4319313   0.5680353   0.25661844  0.4562481  -0.49407065
 -0.3386588   0.27295756 -0.1279482  -0.08139388 -0.27554268 -0.20281184
  0.22055537 -0.8774142   0.6561442  -0.66929215 -0.10679027  0.18666504
 -0.547993    0.7109107   0.9624165  -0.51227903 -0.03076768 -0.59558177
 -0.22885568  0.56230384  0.17289296  0.01146731 -0.99664795  0.4141445
 -0.1369941  -0.3166605  -0.36577192  0.26997894 -0.36139053 -0.09298746
  0.6922673   0.6842534   0.11200828 -0.10805236  0.59492356 -0.6782107
 -0.58614314 -0.05911744 -0.63365024 -0.09294061 -0.085

In [24]:
# Find similar words
similar_words = model.wv.most_similar('mobile', topn=15)
for i in similar_words:
    print(i)

('launchers', 0.9012715220451355)
('grenades', 0.8995427489280701)
('rifles', 0.895071268081665)
('lamp', 0.895065188407898)
('steel', 0.8931792378425598)
('restaurants', 0.8930208086967468)
('sewer', 0.8914008736610413)
('rockets', 0.8899285197257996)
('confiscated', 0.8865554332733154)
('laboratory', 0.8859495520591736)
('controls', 0.8857496976852417)
('storage', 0.8854281902313232)
('squatters', 0.8835244178771973)
('cart', 0.8833296298980713)
('installing', 0.8817546963691711)
